# Netflix Business Solution

In [9]:
# Import Libaries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mysql.connector

In [10]:
# Establish a connection to the MySQL database
db = mysql.connector.connect(
    host="localhost",
    user="root", 
    password="123456",
    database="netflix_db"
)

# Create a cursor object to interact with the database
cur = db.cursor()

In [3]:
#loading dataset
netflix_data=pd.read_csv('netflix_data_and_image/netflix_titles.csv')

In [4]:
netflix_data

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
...,...,...,...,...,...,...,...,...,...,...,...,...
8802,s8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,"November 20, 2019",2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a..."
8803,s8804,TV Show,Zombie Dumb,NaN,NaN,NaN,"July 1, 2019",2018,TV-Y7,2 Seasons,"Kids' TV, Korean TV Shows, TV Comedies","While living alone in a spooky town, a young g..."
8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...
8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero..."


In [91]:
netflix_data['type'].unique()

array(['Movie', 'TV Show'], dtype=object)

# Queries

### 1. Count the number of Movies vs TV Shows

In [92]:
# Define a query
query = """SELECT 
    type,
    COUNT(*)
FROM netflix_titles
GROUP BY 1;
"""

# Execute the query
cur.execute(query)

# Fetch the result
netflix_data = cur.fetchall()

# Build a DataFrame
netflix_df = pd.DataFrame(netflix_data, columns=[desc[0] for desc in cur.description])
netflix_df.head()

,type,COUNT(*)
0,Movie,6131
1,TV Show,2676


### 2. Find the most common rating for movies and TV shows

In [93]:
# Define a query
query = """
WITH RatingCounts AS (
    SELECT 
        type,
        rating,
        COUNT(*) AS rating_count
    FROM netflix_titles
    GROUP BY type, rating
),
RankedRatings AS (
    SELECT
        type,
        rating,
        rating_count,
        DENSE_RANK() OVER (PARTITION BY type ORDER BY rating_count DESC) AS `rank`
    FROM RatingCounts
)
SELECT 
    type,
    rating AS most_frequent_rating
FROM RankedRatings
WHERE `rank` = 1;

"""

# Execute the query
cur.execute(query)

# Fetch the result
netflix_data = cur.fetchall()

# Build a DataFrame
netflix_df = pd.DataFrame(netflix_data,columns=[desc[0] for desc in cur.description])
netflix_df.head()

,type,most_frequent_rating
0,Movie,TV-MA
1,TV Show,TV-MA


### 3. List all movies released in a specific year (e.g., 2020)

In [31]:
query="""
SELECT title
FROM netflix_titles
WHERE type = 'movie' AND release_year = 2020;
"""

# Execute the query
cur.execute(query)

# Fetch the result
netflix_data = cur.fetchall()

# Build a DataFrame
netflix_df = pd.DataFrame(netflix_data,columns = ["title"])
netflix_df

,title
0,Dick Johnson Is Dead
1,Europe's Most Dangerous Man: Otto Skorzeny in ...
2,Tughlaq Durbar
3,Omo Ghetto: the Saga
4,Shadow Parties
...,...
512,All the Freckles in the World
513,Ghost Stories
514,#cats_the_mewvie
515,Norm of the North: Family Vacation


### 4. Find the top 5 countries with the most content on Netflix

In [96]:
query="""
SELECT country, COUNT(*) as total_content
FROM (
    SELECT TRIM(SUBSTRING_INDEX(SUBSTRING_INDEX(country, ',', 1), ',', -1)) AS country FROM netflix_titles WHERE country IS NOT NULL
    UNION ALL
    SELECT TRIM(SUBSTRING_INDEX(SUBSTRING_INDEX(country, ',', 2), ',', -1)) FROM netflix_titles WHERE CHAR_LENGTH(country) - CHAR_LENGTH(REPLACE(country, ',', '')) >= 1
    UNION ALL
    SELECT TRIM(SUBSTRING_INDEX(SUBSTRING_INDEX(country, ',', 3), ',', -1)) FROM netflix_titles WHERE CHAR_LENGTH(country) - CHAR_LENGTH(REPLACE(country, ',', '')) >= 2
    UNION ALL
    SELECT TRIM(SUBSTRING_INDEX(SUBSTRING_INDEX(country, ',', 4), ',', -1)) FROM netflix_titles WHERE CHAR_LENGTH(country) - CHAR_LENGTH(REPLACE(country, ',', '')) >= 3
    UNION ALL
    SELECT TRIM(SUBSTRING_INDEX(SUBSTRING_INDEX(country, ',', 5), ',', -1)) FROM netflix_titles WHERE CHAR_LENGTH(country) - CHAR_LENGTH(REPLACE(country, ',', '')) >= 4
    UNION ALL
    SELECT TRIM(SUBSTRING_INDEX(SUBSTRING_INDEX(country, ',', 6), ',', -1)) FROM netflix_titles WHERE CHAR_LENGTH(country) - CHAR_LENGTH(REPLACE(country, ',', '')) >= 5
) AS country_list
WHERE country IS NOT NULL
GROUP BY country
ORDER BY total_content DESC
LIMIT 5;

"""

# Execute the query
cur.execute(query)

# Fetch the result
netflix_data = cur.fetchall()

# Build a DataFrame (MySQL cursor has a description attribute that contains column names)
netflix_df = pd.DataFrame(netflix_data, columns=[desc[0] for desc in cur.description])

# Display the DataFrame
netflix_df


,country,total_content
0,United States,3686
1,India,1045
2,United Kingdom,804
3,Canada,445
4,France,391


### 5. Identify the longest movie

In [13]:
query="""
SELECT 
    title, duration
FROM 
    netflix_titles
WHERE 
    type = 'Movie'
ORDER BY 
    CAST(SUBSTRING_INDEX(duration, ' ', 1) AS UNSIGNED) DESC;
"""
# Execute the query
cur.execute(query)

# Fetch the result
netflix_data = cur.fetchall()

# Build a DataFrame 
netflix_df = pd.DataFrame(netflix_data, columns=[desc[0] for desc in cur.description])

# Display the DataFrame
netflix_df

,title,duration
0,Black Mirror: Bandersnatch,312 min
1,Headspace: Unwind Your Mind,273 min
2,The School of Mischief,253 min
3,No Longer kids,237 min
4,Lock Your Girls In,233 min
...,...,...
6126,Sol Levante,5 min
6127,Silent,3 min
6128,Louis C.K. 2017,None
6129,Louis C.K.: Hilarious,None


### 6. Find content added in the last 5 years

In [17]:
query="""
SELECT date_added
FROM netflix_titles
WHERE STR_TO_DATE(date_added, '%M %d, %Y') >= CURDATE() - INTERVAL 5 YEAR;
"""
# Execute the query
cur.execute(query)

# Fetch the result
netflix_data = cur.fetchall()

# Build a DataFrame 
netflix_df = pd.DataFrame(netflix_data, columns=[desc[0] for desc in cur.description])

# Display the DataFrame
netflix_df

,date_added
0,"September 25, 2021"
1,"September 24, 2021"
2,"September 24, 2021"
3,"September 24, 2021"
4,"September 24, 2021"
...,...
4075,"November 20, 2019"
4076,"December 31, 2019"
4077,"November 20, 2019"
4078,"November 1, 2019"


### 7. Find all the movies/TV shows by director 'Rajiv Chilaka'!

In [20]:
query="""
SELECT type, title
FROM netflix_titles
WHERE FIND_IN_SET('Rajiv Chilaka', director) > 0;
"""
# Execute the query
cur.execute(query)

# Fetch the result
netflix_data = cur.fetchall()

# Build a DataFrame 
netflix_df = pd.DataFrame(netflix_data, columns=[desc[0] for desc in cur.description])

# Display the DataFrame
netflix_df

,type,title
0,Movie,Chhota Bheem - Neeli Pahaadi
1,Movie,Chhota Bheem & Ganesh
2,Movie,Chhota Bheem & Krishna: Mayanagari
3,Movie,Chhota Bheem & Krishna: Pataliputra- City of t...
4,Movie,Chhota Bheem And The Broken Amulet
5,Movie,Chhota Bheem And The Crown of Valhalla
6,Movie,Chhota Bheem and the Incan Adventure
7,Movie,Chhota Bheem and The ShiNobi Secret
8,Movie,Chhota Bheem Aur Hanuman
9,Movie,Chhota Bheem aur Krishna


### 8. List all TV shows with more than 5 seasons

In [24]:
query="""
SELECT type, title, duration
FROM netflix_titles
WHERE type = 'TV Show'
  AND CAST(SUBSTRING_INDEX(duration, ' ', 1) AS UNSIGNED) > 5;
"""

# Execute the query
cur.execute(query)

# Fetch the result
netflix_data = cur.fetchall()

# Build a DataFrame 
netflix_df = pd.DataFrame(netflix_data, columns=[desc[0] for desc in cur.description])

# Display the DataFrame
netflix_df

,type,title,duration
0,TV Show,The Great British Baking Show,9 Seasons
1,TV Show,Nailed It,6 Seasons
2,TV Show,Numberblocks,6 Seasons
3,TV Show,Saved by the Bell,9 Seasons
4,TV Show,Lucifer,6 Seasons
...,...,...,...
94,TV Show,The Andy Griffith Show,8 Seasons
95,TV Show,The L Word,6 Seasons
96,TV Show,The Office (U.S.),9 Seasons
97,TV Show,The West Wing,7 Seasons


### 9. Count the number of content items in each genre

In [25]:
query="""
SELECT 
    listed_in,
    COUNT(*) AS total_content
FROM netflix_titles
GROUP BY listed_in;
"""

# Execute the query
cur.execute(query)

# Fetch the result
netflix_data = cur.fetchall()

# Build a DataFrame 
netflix_df = pd.DataFrame(netflix_data, columns=[desc[0] for desc in cur.description])

# Display the DataFrame
netflix_df

,listed_in,total_content
0,Documentaries,359
1,"International TV Shows, TV Dramas, TV Mysteries",26
2,"Crime TV Shows, International TV Shows, TV Act...",18
3,"Docuseries, Reality TV",16
4,"International TV Shows, Romantic TV Shows, TV ...",94
...,...,...
509,"Action & Adventure, Comedies, Horror Movies",1
510,"Classic & Cult TV, Crime TV Shows, TV Dramas",1
511,"Action & Adventure, Documentaries, Sports Movies",1
512,"International Movies, LGBTQ Movies, Romantic M...",3


### 10.Find each year and the average numbers of content release in India on netflix. return top 5 year with highest avg content release!

In [26]:
query="""
SELECT 
    country,
    release_year,
    COUNT(show_id) AS total_release,
    ROUND(
        COUNT(show_id) / 
        (SELECT COUNT(show_id) FROM netflix_titles WHERE country = 'India') * 100, 2
    ) AS avg_release
FROM netflix_titles
WHERE country = 'India'
GROUP BY country, release_year
ORDER BY avg_release DESC
LIMIT 5;
"""

# Execute the query
cur.execute(query)

# Fetch the result
netflix_data = cur.fetchall()

# Build a DataFrame 
netflix_df = pd.DataFrame(netflix_data, columns=[desc[0] for desc in cur.description])

# Display the DataFrame
netflix_df

,country,release_year,total_release,avg_release
0,India,2017,101,10.39
1,India,2018,94,9.67
2,India,2019,87,8.95
3,India,2020,75,7.72
4,India,2016,73,7.51


### 11. List all movies that are documentaries

In [36]:
query="""
SELECT type, title
FROM netflix_titles
WHERE listed_in LIKE '%Documentaries';
"""

# Execute the query
cur.execute(query)

# Fetch the result
netflix_data = cur.fetchall()

# Build a DataFrame 
netflix_df = pd.DataFrame(netflix_data, columns=[desc[0] for desc in cur.description])

# Display the DataFrame
netflix_df

,type,title
0,Movie,Dick Johnson Is Dead
1,Movie,My Heroes Were Cowboys
2,Movie,Final Account
3,Movie,"Bob Ross: Happy Accidents, Betrayal & Greed"
4,Movie,Lady Boss: The Jackie Collins Story
...,...,...
378,Movie,What Makes a Psychopath?
379,Movie,Why Are We Getting So Fat?
380,Movie,Why Knot
381,Movie,Why We Fight: The Battle of Russia


### 12. Find all content without a director

In [38]:
query="""
SELECT * 
FROM netflix_titles
WHERE director IS NULL;
"""
# Execute the query
cur.execute(query)

# Fetch the result
netflix_data = cur.fetchall()

# Build a DataFrame 
netflix_df = pd.DataFrame(netflix_data, columns=[desc[0] for desc in cur.description])

# Display the DataFrame
netflix_df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s2,TV Show,Blood & Water,None,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
1,s4,TV Show,Jailbirds New Orleans,None,None,None,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
2,s5,TV Show,Kota Factory,None,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
3,s11,TV Show,"Vendetta: Truth, Lies and The Mafia",None,None,None,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, Docuseries, International TV S...","Sicily boasts a bold ""Anti-Mafia"" coalition. B..."
4,s15,TV Show,Crime Stories: India Detectives,None,None,None,"September 22, 2021",2021,TV-MA,1 Season,"British TV Shows, Crime TV Shows, Docuseries",Cameras following Bengaluru police on the job ...


### 13. Find how many movies actor 'Salman Khan' appeared in last 10 years!

In [44]:
query="""
SELECT title,cast,date_added,re
FROM netflix_titles
WHERE cast LIKE '%Salman Khan%'
  AND release_year > YEAR(CURDATE()) - 10;
"""
# Execute the query
cur.execute(query)

# Fetch the result
netflix_data = cur.fetchall()

# Build a DataFrame 
netflix_df = pd.DataFrame(netflix_data, columns=[desc[0] for desc in cur.description])

# Display the DataFrame
netflix_df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s2340,Movie,Prem Ratan Dhan Payo,Sooraj R. Barjatya,"Salman Khan, Sonam Kapoor, Anupam Kher, Neil N...",India,"June 25, 2020",2015,TV-14,164 min,"Action & Adventure, Dramas, International Movies",While recovering from an assassination attempt...
1,s2757,Movie,Paharganj,Rakesh Ranjan Kumar,"Lorena Franco, Bijesh Jayarajan, Neet Chowdhar...",India,"March 31, 2020",2019,TV-MA,107 min,"Dramas, Independent Movies, International Movies","In the underbelly of a Delhi neighborhood, a w..."


### 14. Find the top 10 actors who have appeared in the highest number of movies produced in India.

In [48]:
query="""SELECT actor, COUNT(*) AS actor_count
FROM (
    SELECT TRIM(SUBSTRING_INDEX(SUBSTRING_INDEX(cast, ',', numbers.n), ',', -1)) AS actor
    FROM netflix_titles
    INNER JOIN (
        SELECT 1 AS n UNION ALL SELECT 2 UNION ALL SELECT 3 UNION ALL SELECT 4
        UNION ALL SELECT 5 UNION ALL SELECT 6 UNION ALL SELECT 7 UNION ALL SELECT 8
        UNION ALL SELECT 9 UNION ALL SELECT 10
    ) numbers ON CHAR_LENGTH(cast) - CHAR_LENGTH(REPLACE(cast, ',', '')) >= numbers.n - 1
    WHERE country = 'India'
) AS actor_list
WHERE actor != ''
GROUP BY actor
ORDER BY actor_count DESC
LIMIT 10;
"""

# Execute the query
cur.execute(query)

# Fetch the result
netflix_data = cur.fetchall()

# Build a DataFrame 
netflix_df = pd.DataFrame(netflix_data, columns=[desc[0] for desc in cur.description])

# Display the DataFrame
netflix_df

,actor,actor_count
0,Anupam Kher,39
1,Shah Rukh Khan,32
2,Naseeruddin Shah,29
3,Akshay Kumar,28
4,Amitabh Bachchan,28
5,Paresh Rawal,27
6,Om Puri,26
7,Kareena Kapoor,24
8,Boman Irani,22
9,Ajay Devgn,20


### 15.Categorize the content based on the presence of the keywords 'kill' and 'violence' in the description field. Label content containing these keywords as 'Bad' and all other content as 'Good'. Count how many items fall into each category.

In [50]:
query="""
SELECT 
    category,
    COUNT(*) AS content_count
FROM (
    SELECT 
        CASE 
            WHEN description LIKE '%kill%' OR description LIKE '%violence%' THEN 'Bad'
            ELSE 'Good'
        END AS category
    FROM netflix_titles
) AS categorized_content
GROUP BY category;
"""

# Execute the query
cur.execute(query)

# Fetch the result
netflix_data = cur.fetchall()

# Build a DataFrame 
netflix_df = pd.DataFrame(netflix_data, columns=[desc[0] for desc in cur.description])

# Display the DataFrame
netflix_df

,category,content_count
0,Good,8465
1,Bad,342
